The objetive of this project is to identify a person from the way they type on keyboard. The time intervals pattern between keystrockes could constitute a "finguerprint" that is characteristic to an indivitual person. I will be testing here is this concept on two indivuals, trying to predic who is typing.
I first collected the raw data using a keylogger python code, this provided me with the two files: keylog1.txt and keylog2.tex, each belong to a typer. I will be using the keras library:

In [8]:
import keras
import numpy as np

We need first to clean the data. each line of the files is composed a date, a time and a character, for example: " 2021-03-07 22:38:02,689: 'h' " is the line generated by typing 'h'. The goal of the cleaning code, is to generate a list of four letters words, each two consequetive letters are seperated by their time intervals. So our training data will be a list of 7 elements arrays. a letter will be represented by a number which is its alphabetic order.

In [13]:
f = open("keyLog1.txt", encoding="utf8", errors='ignore')
Training1 = []
e = 0
N = np.zeros(7)
for x in f:
    if len(x[20:]) > 9:
        e = 0
        N = np.zeros(8)
    elif (len(x[20:]) == 9) and ( 96 < ord(x[26:27]) < 123 ):
        if (e == 0):
            I = int(x[20:23])
        if (e < 4):
            e = e + 1
        if (e == 1):
            N[0] = ord(x[26:27]) - 96
        if(5>e>1):
            N[2*e-2]= ord(x[26:27]) - 96
            N[2*e-3]= int(x[20:23]) - I + 999*bool(int(x[20:23]) < I)
            I = int(x[20:23])
        if(e == 4):
            Training1.append(list(N))
            e = 5

In [14]:
np.shape(Training1)

(373, 8)

The cleaning code for the second file:

In [16]:
f = open("keyLog2.txt", encoding="utf8", errors='ignore')
Training2 = []
e = 0
N = np.zeros(7)
for x in f:
    if len(x[20:]) > 9:
        e = 0
        N = np.zeros(8)
    elif (len(x[20:]) == 9) and ( 96 < ord(x[26:27]) < 123 ):
        if (e == 0):
            I = int(x[20:23])
        if (e < 4):
            e = e + 1
        if (e == 1):
            N[0] = ord(x[26:27]) - 96
        if(5>e>1):
            N[2*e-2]= ord(x[26:27]) - 96
            N[2*e-3]= int(x[20:23]) - I + 999*bool(int(x[20:23]) < I)
            I = int(x[20:23])
        if(e == 4):
            Training2.append(list(N))
            e = 5

In [17]:
np.shape(Training2)

(422, 8)

Now we combine the two list of data and then we create as labels list.


In [ ]:
Training = list(Training1)+list(Training2) # Combining the two lists
Labels = list(np.ones(len(Training1))) + list(np.zeros(len(Training2)))
print(np.shape(Training))
print(np.shape(Labels))

(795, 8)
(795,)


Now we shuffle the data list and lable list in the same way. then we choose 10% of each list to serve as test data.

In [ ]:
p = np.random.permutation(len(Training))
train_data = [Training[p[x]] for x in range(len(Training))][:int(0.9*len(Training))]
train_labels = [Labels[p[x]] for x in range(len(Training))][:int(0.9*len(Training))]
test_data = [Training[p[x]] for x in range(len(Training))][int(0.9*len(Training)):]
test_labels = [Labels[p[x]] for x in range(len(Training))][int(0.9*len(Training)):]
print(np.shape(train_data))
print(np.shape(train_labels))
print(np.shape(test_data))
print(np.shape(test_labels))

(715, 8)
(715,)
(80, 8)
(80,)


In [ ]:
from keras import models 
from keras import layers

In [ ]:
We choose the parameters that are adapted to our problem:

In [ ]:
model = models.Sequential() 
model.add(layers.Dense(70, activation='relu', input_shape=(8,))) 
model.add(layers.Dense(2, activation='sigmoid'))
model.compile(optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy'])

In [ ]:
from keras.utils import to_categorical

In [ ]:
train_labels = to_categorical(train_labels) 
test_labels = to_categorical(test_labels)

In [ ]:
np.shape(train_data)

(715, 8)

In [ ]:
history=model.fit(np.array(train_data), np.array(train_labels), epochs=150, batch_size=715)

Epoch 1/150
1/1 [==============================] - 0s 343ms/step - loss: 26.0827 - accuracy: 0.5357
Epoch 2/150
1/1 [==============================] - 0s 5ms/step - loss: 24.3193 - accuracy: 0.5371
Epoch 3/150
1/1 [==============================] - 0s 6ms/step - loss: 22.5732 - accuracy: 0.5371
Epoch 4/150
1/1 [==============================] - 0s 5ms/step - loss: 20.8739 - accuracy: 0.5357
Epoch 5/150
1/1 [==============================] - 0s 5ms/step - loss: 19.2688 - accuracy: 0.5315
Epoch 6/150
1/1 [==============================] - 0s 7ms/step - loss: 17.7866 - accuracy: 0.5301
Epoch 7/150
1/1 [==============================] - 0s 6ms/step - loss: 16.4308 - accuracy: 0.5287
Epoch 8/150
1/1 [==============================] - 0s 5ms/step - loss: 15.1442 - accuracy: 0.5273
Epoch 9/150
1/1 [==============================] - 0s 6ms/step - loss: 13.8838 - accuracy: 0.5273
Epoch 10/150
1/1 [==============================] - 0s 5ms/step - loss: 12.6231 - accuracy: 0.5329
Epoch 11/150
1/1 

In [ ]:
results = model.evaluate(np.array(test_data), np.array(test_labels),batch_size=715)
print("test loss, test acc:", results)

1/1 [==============================] - 0s 16ms/step - loss: 0.6818 - accuracy: 0.7750
test loss, test acc: [0.6817992329597473, 0.7749999761581421]


We have a final accuracy around 75%. To obtain higher accuracy, we need more training data.
